In [6]:
import pandas as pd
import csv
import numpy as np
from trading_calendars import get_calendar
from zipline.utils.calendars import TradingCalendar
from zipline.utils.tradingcalendar import get_trading_days
from pandas import Timestamp


#%%
#This is the process to get the correct trading days, which will then be use in the Bundle Ingest process to index the days
trading_days = get_trading_days(start=Timestamp('2018-01-01 00:00:00+0000', tz='UTC'), 
                                end=Timestamp('2021-08-24 02:21:29.456875+0000')
                                ).date.astype(str)

#%%
# Create a CSV file to set the Zipline Bundle Ingest process against
with open('trading_calendar.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(trading_days)

In [7]:
import pandas as pd
import csv
import numpy as np
from trading_calendars import get_calendar
from zipline.utils.calendars import TradingCalendar
from zipline.utils.tradingcalendar import get_trading_days
from pandas import Timestamp


#%%
#This is the process to get the correct trading days, which will then be use in the Bundle Ingest process to index the days
trading_days = get_trading_days(start=Timestamp('2018-01-01 00:00:00+0000', tz='UTC'), 
                                end=Timestamp('2021-08-24 00:00:00+0000')
                                ).date.astype(str)

#%%
# Create a CSV file to set the Zipline Bundle Ingest process against
with open('trading_calendar.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(trading_days)

In [8]:
import csv
import numpy as np
import pandas as pd
import os

#%%
# Get the Trading Calendar, run from a zip35 code environment
with open('trading_calendar.csv') as f:
    reader = csv.reader(f)
    data = list(reader)

arr = np.array(data)
trading_days = arr.ravel()

In [10]:
def format_bundle(indir, outdir):
       
    count = 0
    for f in os.listdir(indir): # For Production
    # for f in ['AAPL.csv']:  # For Testing
        
        df = pd.read_csv('{}/{}'.format(indir, f), index_col='date')
        
        # I need to add some logic here to force the number of rows to equal the number of expected trading days
        df = df.reindex(trading_days)

        # Export it in the csvdir format needed for the ziplien bundle ingestion process
        df.reset_index(inplace=True)
        
        # Check if there is there is any divident, if not make it zero
        if not 'dividend' in df.columns:
            df['dividend'] = 0.0
        
        #df = df[['date', 'open_adj', 'high_adj', 'low_adj', 'close_adj', 'volume_adj', 'dividend']]
        df['dividend'].fillna(0.00, inplace=True)
        df['ratio'] = 1  # Since I alread did all the adjusting
        
        #df.rename(columns={'open_adj': 'open',
         #                  'high_adj': 'high',
         #                  'low_adj': 'low',
         #                  'close_adj': 'close',
          #                 'volume_adj': 'volume'
          #                 }, inplace=True)
        
        # Round the numbers in the dataframe
        df = df.round({'open':4,
                  'high':4,
                  'low':4,
                  'close':4,
                  'volume':1,
                  'dividend':2})

        df.to_csv('{}/{}'.format(outdir, f), index=False)

        count += 1
        
    return ('{} files was adjusted'.format(count))


#%%  Execute the function to format the files
format_bundle('data/bars_adj', 'data/csvs/daily')

'11 files was adjusted'